In [1]:
import pandas as pd
from sqlalchemy import create_engine
import getpass

pd.options.display.max_rows = 4

### Connect to databases.

MySQL is the old database hosted on Dreamhost.

Postgres is the new database on localhost.

In [2]:
password = getpass.getpass('Database password: ')

Database password: ········


In [3]:
engine_postgres = create_engine('postgres://aubreymoore:{}@localhost/import_test'.format(password))
engine_mysql = create_engine('mysql+pymysql://aubreymoore:{}@mysql.guaminsects.net/pestlist'.format(password))

print 'postgres'
print engine_postgres.table_names()
print
print 'mysql'
print engine_mysql.table_names()
#pd.read_sql_table('taxon', engine)

postgres
[u'name', u'auth_event', u'auth_membership', u'auth_permission', u'auth_user', u'bugwood', u'auth_cas', u'factsheet', u'pacific_pests', u'syn', u'synonym2', u'vernacular', u'name_type', u'image', u'biblio', u'biblio_type', u'associate', u'geo', u'geo_biblio', u'distribution', u'taxon', u'auth_group']

mysql
[u'associate2', u'auth_cas', u'auth_event', u'auth_group', u'auth_membership', u'auth_permission', u'auth_user', u'bugwood', u'extracted_names', u'factsheet', u'geo', u'location_recode', u'pacific_pests', u'resolved_names', u'syn', u'synonym2', u'taxon2', u'vernacular']


### Create a csv import file for the **associate** table.

In [4]:
df_associate2 = pd.read_sql_table('associate2', engine_mysql)
df_associate2

,id,t1,t2,type,note
0,1,7,326,plant host|herbivore,None
1,2,7,342,plant host|herbivore,None
...,...,...,...,...,...
929,930,319,1037,plant host|herbivore,None
930,931,319,752,plant host|herbivore,None


In [5]:
df_taxon2 = pd.read_sql_table('taxon2', engine_mysql)
df_taxon2

,id,tid,parent_tid,name,trank,lineage
0,1,6,None,Plantae,kingdom,Plantae
1,2,7707728,6,Tracheophyta,phylum,Plantae|Tracheophyta
...,...,...,...,...,...,...
804,6107,3177662,5014,Eugenia,genus,Plantae|Tracheophyta|Magnoliopsida|Myrtales|My...
805,6108,7907394,3177662,Eugenia malaccensis,species,Plantae|Tracheophyta|Magnoliopsida|Myrtales|My...


In [6]:
# Get names for t1 and t2 in associate2 table
df = df_associate2.merge(right=df_taxon2, left_on='t1', right_on='id')
df = df.merge(right=df_taxon2, left_on='t2', right_on='id')
df = df[['name_x', 'name_y']]

# Replace names with ids from taxon table
df_taxon = pd.read_sql_table('taxon', engine_postgres)
df = df.merge(df_taxon, left_on='name_x', right_on='name')
df = df.merge(df_taxon, left_on='name_y', right_on='name')
df = df[['id_x', 'id_y']]
df.columns = ['associate.taxon1', 'associate.taxon2']

# Write CSV file and display data frame
df.to_csv('associate.csv', index=False)
df

,associate.taxon1,associate.taxon2
0,2315,1932
1,2429,1932
...,...,...
929,2412,1858
930,2412,2191


### Create a csv import file for the **names** table.

In [7]:
pd.read_sql_table('syn', engine_mysql)

,id,taxon,synonym
0,1,299,Lycopersicon esculentum
1,2,353,Trigonops sp.
...,...,...,...
38,40,3031,Cyrtopeltis tenuis
39,41,6096,Saissetia ?oleae


In [8]:
pd.read_sql_table('synonym2', engine_mysql)

,id,t1,name,source
0,1,1954,"Epilachna Dejean, 1835",http://api.gbif.org/v1/species/1043619/synonyms
1,2,2829,"Hasticollinum Kaszab, 1939",http://api.gbif.org/v1/species/1045882/synonyms
...,...,...,...,...
3170,3171,896,Podopidae,http://api.gbif.org/v1/species/9650/synonyms
3171,3172,890,Plataspididae,http://api.gbif.org/v1/species/9652/synonyms


In [9]:
pd.read_sql_table('vernacular', engine_mysql)

,id,t1,language,source,name
0,139,320,eng,http://api.gbif.org/v1/species/1/vernacularNames,Animals
1,140,1948,eng,http://api.gbif.org/v1/species/1048253/vernacu...,tobacco flea beetle
...,...,...,...,...,...
858,2649,2467,,http://api.gbif.org/v1/species/9717/vernacular...,nolid moths
859,2650,1690,eng,http://api.gbif.org/v1/species/999/vernacularN...,termites


In [10]:
pd.options.display.max_rows = 100
pd.read_sql_table('name_type', engine_postgres)

,id,name_type
0,1,GBIF ID
1,2,accepted scientific name
2,3,scientific synonym
3,4,vernacular
4,5,scientific mispelling
5,6,vernacular mispelling
6,7,vernacular (English)
7,8,vernacular (Chamorro)


In [11]:
pd.options.display.max_rows = 4
df_taxon2

,id,tid,parent_tid,name,trank,lineage
0,1,6,None,Plantae,kingdom,Plantae
1,2,7707728,6,Tracheophyta,phylum,Plantae|Tracheophyta
...,...,...,...,...,...,...
804,6107,3177662,5014,Eugenia,genus,Plantae|Tracheophyta|Magnoliopsida|Myrtales|My...
805,6108,7907394,3177662,Eugenia malaccensis,species,Plantae|Tracheophyta|Magnoliopsida|Myrtales|My...


In [12]:
df_taxon

,id,parent_id,name,lineage,trank
0,1625,None,Animalia,Animalia,kingdom
1,1626,1625,Arthropoda,Animalia|Arthropoda,phylum
...,...,...,...,...,...
803,2429,2427,Solanum melongena,Plantae|Tracheophyta|Magnoliopsida|Solanales|S...,species
804,2430,2427,Solanum tuberosum,Plantae|Tracheophyta|Magnoliopsida|Solanales|S...,species


 **df_lookup** is a lookup table relating the old index, **tid**, which is usually a **GBIF ID**, to **name.taxon**, which is the FK in the **name** table. Note that not all taxa have GBIF IDs. These have have alphanumeric ids.

In [54]:
df_lookup = df_taxon.merge(df_taxon2, on='name')
df_lookup = df_lookup[['id_x','tid']]
df_lookup.columns = ['new_id', 'old_id']
df_lookup

,new_id,old_id
0,1625,1
1,1626,54
...,...,...
804,2429,2930617
805,2430,2930262


### name_GBIF.csv

In [50]:
df_GBIF = df_lookup
df_GBIF['name.name_type'] = 1
df_GBIF.columns = ['name.taxon', 'name.name', 'name.name_type']

# Write CSV file and display data frame
df.to_csv('name_GBIF.csv', index=False)
df_GBIF

,name.taxon,name.name,name.name_type
0,1625,1,1
1,1626,54,1
...,...,...,...
804,2429,2930617,1
805,2430,2930262,1


### name_synonym.csv

In [55]:
df_synonym = pd.read_sql_table('synonym2', engine_mysql)
df_synonym

,id,t1,name,source
0,1,1954,"Epilachna Dejean, 1835",http://api.gbif.org/v1/species/1043619/synonyms
1,2,2829,"Hasticollinum Kaszab, 1939",http://api.gbif.org/v1/species/1045882/synonyms
...,...,...,...,...
3170,3171,896,Podopidae,http://api.gbif.org/v1/species/9650/synonyms
3171,3172,890,Plataspididae,http://api.gbif.org/v1/species/9652/synonyms


In [56]:
df_synonym.merge(right=df_lookup, left_on='t1', right_on='old_id')

,id,t1,name,source,new_id,old_id


### name_vernacular.csv

In [34]:
df_vernacular = pd.read_sql_table('vernacular', engine_mysql)
df_vernacular.merge(df_lookup, left_on='t1', right_on='name.taxon')


,id,t1,language,source,name,name.taxon,tid
0,140,1948,eng,http://api.gbif.org/v1/species/1048253/vernacu...,tobacco flea beetle,1948,7908854
1,141,2102,eng,http://api.gbif.org/v1/species/1080274/vernacu...,Mango flower beetle,2102,2007333
...,...,...,...,...,...,...,...
116,2635,2213,eng,http://api.gbif.org/v1/species/9417/vernacular...,swallowtail butterfly,2213,8868
117,2650,1690,eng,http://api.gbif.org/v1/species/999/vernacularN...,termites,1690,AM0018
